# Étape 1 - Configurer MLflow
MLflow est disponible sur PyPI. Si vous ne l'avez pas encore installé sur votre système, vous pouvez l'installer avec :

Ensuite, suivez les instructions du guide d'installation de MLflow pour configurer MLflow.

Si vous souhaitez démarrer très rapidement, exécutez le code suivant dans une cellule de notebook :

In [ ]:
import mlflow

mlflow.set_experiment("MLflow Quickstart")

# Étape 2 - Préparer les donnees
Avant d'entraîner notre premier modèle, préparons les données d'entraînement et les hyperparamètres du modèle.

In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Étape 3 - Entraîner un modèle avec l'auto-enregistrement
Dans cette étape, nous entraînons le modèle sur les données d'entraînement chargées à l'étape précédente, puis nous enregistrons le modèle et ses métadonnées dans MLflow. La méthode la plus simple consiste à utiliser la fonction d'enregistrement automatique de MLflow .

In [ ]:
import mlflow

# Enable autologging for scikit-learn
mlflow.sklearn.autolog()

# Just train the model normally
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

Avec une seule ligne de code supplémentaire mlflow.sklearn.autolog(), vous bénéficiez désormais du meilleur des deux mondes : vous pouvez vous concentrer sur l’entraînement du modèle, et MLflow s’occupe du reste :

Enregistrement du modèle entraîné.
Enregistrement des indicateurs de performance du modèle pendant l'entraînement, tels que l'exactitude, la précision et la courbe AUC.
Enregistrement des valeurs des hyperparamètres utilisés pour entraîner le modèle.
Suivi des métadonnées telles que le format des données d'entrée, l'utilisateur, l'horodatage, etc.
Pour en savoir plus sur la journalisation automatique et les bibliothèques prises en charge, consultez la documentation sur la journalisation automatique .

# Étape 5 - Saisir manuellement
Maintenant que nous avons appris à consigner l'entraînement d'un modèle avec la journalisation automatique de MLflow, passons à l'étape suivante et apprenons à consigner manuellement un modèle et ses métadonnées. Cette méthode est utile lorsque vous souhaitez un contrôle plus précis du processus de journalisation.

Voici les étapes que nous allons suivre :

- Lancez un contexte d'exécution MLflow pour démarrer une nouvelle exécution dans laquelle nous enregistrerons le modèle et les métadonnées.
- Entraînez et testez le modèle.
- Paramètres du modèle de journalisation et indicateurs de performance .
- Étiquetez la course pour la retrouver facilement.

In [ ]:
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Train the model
    lr = LogisticRegression(**params)
    lr.fit(X_train, y_train)

    # Log the model
    model_info = mlflow.sklearn.log_model(sk_model=lr, name="iris_model")

    # Predict on the test set, compute and log the loss metric
    y_pred = lr.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mlflow.log_metric("accuracy", accuracy)

    # Optional: Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

# Étape 6 - Recharger le modèle pour l'inférence 
Après avoir enregistré le modèle, nous pouvons effectuer l'inférence en procédant comme suit :

- Chargement du modèle en utilisant la pyfuncversion de MLflow.
- Exécution de la prédiction sur de nouvelles données en utilisant le modèle chargé.

In [ ]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.sklearn.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

result[:4]